# Vehicle Classification

## Evaluate your model


In [1]:
# Importing the necessary libraries
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import subclass_compress
from utils import utils
from sklearn.metrics import accuracy_score, classification_report
from densenet_model import densenet121_model

## Experiment settings

Set here the two variables in the following way:

- **CONFIG_YML:** assign the path to the config.yml file used for the experiment you want to evaluate
- **WEIGHTS:** assign the path to the model weights (.h5 file) you want to evaluate


In [6]:
# Getting the yaml and weights files to be used
CONFIG_YML = "/home/app/src/densenet_weights/w_006/config_densenet.yml"
WEIGHTS = "/home/app/src/densenet_weights/w_006/model.108-0.8851.h5"

In [7]:
# Showing the yaml configuration file
config = utils.load_config(CONFIG_YML)
config

{'seed': 123,
 'data': {'directory': '/data_volume/dataset/sample_classifier_images/train',
  'labels': 'inferred',
  'label_mode': 'categorical',
  'validation_split': 0.2,
  'image_size': [224, 224],
  'batch_size': 32},
 'model': {'weights': '/home/app/src/densenet_weights/best_head_w/model.143-4.9448.h5',
  'input_shape': [224, 224, 3],
  'classes': 836,
  'dropout_rate': 0.5,
  'data_aug_layer': {'random_flip': {'mode': 'horizontal'},
   'random_rotation': {'factor': 0.2},
   'random_zoom': {'height_factor': 0.05, 'width_factor': 0.05},
   'random_brightness': {'factor': 0.25}},
  'freeze': []},
 'compile': {'optimizer': {'adam': {'learning_rate': 0.001}},
  'loss': 'categorical_crossentropy',
  'metrics': ['accuracy']},
 'fit': {'epochs': 250,
  'callbacks': {'model_checkpoint': {'filepath': '/home/app/src/densenet_weights/w_006/model.{epoch:02d}-{val_loss:.4f}.h5',
    'save_best_only': True},
   'tensor_board': {'log_dir': '/home/app/src/densenet_weights/w_006/logs'},
   'reduc

In [8]:
# Check if there are any mistakes with the information

MODEL_CLASSES = utils.get_class_names(config)

if len(MODEL_CLASSES) != config['model']['classes']:
    raise ValueError(
        "Number of classes doesn't match between your model "
        "and your data!"
    )

_dirname, _ = os.path.split(config['data']['directory'])
TEST_FOLDER = os.path.join(_dirname, 'test')
print(TEST_FOLDER)
if not os.path.exists(TEST_FOLDER):
    raise ValueError("'test' folder not found!")
    
if len(os.listdir(TEST_FOLDER)) != config['model']['classes']:
    raise ValueError(
        "Number of classes doesn't match between your model "
        "and your testing dataset!"
    )
    
if set(os.listdir(TEST_FOLDER)) != set(MODEL_CLASSES):
    raise ValueError(
        "The name of the subfolders inside your test set "
        "doesn't match with the model classes!"
    )

/data_volume/dataset/sample_classifier_images/test


## Load your model

Use `densenet121_model.create_model()` and remember to properly setup the model weights!

Assign the model to the variable `cnn_model`.


In [9]:
# Loading weights of the model and printing the summary
cnn_model = densenet121_model.create_model(weights=WEIGHTS)

print(cnn_model.summary())

2022-07-30 20:13:20.569562: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-30 20:13:20.580059: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-30 20:13:20.580968: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-30 20:13:20.583133: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 224, 224, 3)       0         
                                                                 
 tf.math.truediv (TFOpLambda  (None, 224, 224, 3)      0         
 )                                                               
                                                                 
 tf.nn.bias_add (TFOpLambda)  (None, 224, 224, 3)      0         
                                                                 
 tf.math.truediv_1 (TFOpLamb  (None, 224, 224, 3)      0         
 da)                                                             
                                                                 
 densenet121 (Functional)    (None, 1024)              703750

## Get predictions from testing dataset

In [10]:
# Using the loaded model and the function utils.predict_from_folder() to get 
# the model predictions and the corresponding true labels so we can measure
# the accuracy

predictions, labels, pred_file_paths = utils.predict_from_folder(
    folder=TEST_FOLDER, 
    model=cnn_model, 
    input_size=config["data"]["image_size"], 
    class_names=MODEL_CLASSES,
)

if len(predictions) != len(labels):
    raise ValueError(
        "The lenght of predictions and labels lists doesn't match!"
    )

if not isinstance(predictions[0], str):
    raise ValueError(
        "Model predictions should be represented as string. E.g: 'Acura RL Sedan 2012'"
    )

if not isinstance(labels[0], str):
    raise ValueError(
        "Ground true labels should be represented as string. E.g: 'Acura RL Sedan 2012'"
    )


0it [00:00, ?it/s]2022-07-30 20:13:35.336951: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8200
2022-07-30 20:13:35.654180: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-07-30 20:13:35.654811: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-07-30 20:13:35.654867: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2022-07-30 20:13:35.655694: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-07-30 20:13:35.655964: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
19079it [34:47,  9.14it/s]


Raw Classification Score:

In [11]:
# Printing the classification report

print(classification_report(y_true=labels, y_pred=predictions))


                                   precision    recall  f1-score   support

                abarth_124-spider       0.77      0.56      0.65        18
                       abarth_500       0.35      0.32      0.33        28
                      abarth_500c       0.43      0.36      0.39        25
                       abarth_595       0.31      0.23      0.26        22
          abarth_595-competizione       0.46      0.24      0.32        25
               abarth_595-turismo       0.20      0.50      0.29        12
                      abarth_595c       0.13      0.23      0.17        13
                       abarth_695       0.83      0.53      0.65        19
              abarth_grande-punto       0.75      0.90      0.82        30
                 abarth_punto-evo       0.81      0.48      0.60        27
                   alfa-romeo_147       0.64      0.58      0.61        24
                   alfa-romeo_156       0.38      0.75      0.50        28
                   alfa-

In [12]:
# Calculating the model's accuracy

acc = accuracy_score(y_true=labels, y_pred=predictions)

print(f"Your model accuracy is {acc:.4f}!")

if acc < .3:
    raise ValueError("Your model accuracy is too low :(\nYou can do it better! :)")


Your model accuracy is 0.5607!


#### Now we are merging some vehicle classes that are very similar with each other into one category since they behave more like subclasses. This should improve the test results 

In [260]:
# Merging the classes into similar categories
def compress_sub_classes(models):
    for i in range(len(models)):
        models[i] = subclass_compress.class_mapping(models[i])
    return models
        

In [261]:
# Merge sublasses into main both in labels and predictions
new_predictions = predictions.copy()
new_labels = labels.copy()
new_predictions = compress_sub_classes(new_predictions)
new_labels = compress_sub_classes(new_labels)

In [262]:
# Printing the classification report

print(classification_report(y_true=new_labels, y_pred=new_predictions))

                                precision    recall  f1-score   support

             abarth_124-spider       0.77      0.56      0.65        18
                    abarth_500       0.81      0.84      0.83       125
                    abarth_695       0.83      0.53      0.65        19
           abarth_grande-punto       0.75      0.90      0.82        30
              abarth_punto-evo       0.81      0.48      0.60        27
                alfa-romeo_147       0.64      0.58      0.61        24
                alfa-romeo_156       0.38      0.75      0.50        28
                alfa-romeo_159       0.61      0.74      0.67        27
                alfa-romeo_166       0.46      0.46      0.46        13
                 alfa-romeo_4c       0.86      0.55      0.67        22
             alfa-romeo_giulia       0.89      0.68      0.77        25
          alfa-romeo_giulietta       0.57      0.62      0.59        21
                 alfa-romeo_gt       0.78      0.64      0.71  

In [263]:
# Calculating the model's accuracy

acc = accuracy_score(y_true=new_labels, y_pred=new_predictions)

print(f"Your model accuracy is {acc:.4f}!")


Your model accuracy is 0.6690!
